In [12]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import text as query_text
from sqlalchemy.sql import func
import os

import pandas as pd
import numpy as np

app = Flask(__name__)
# Configure SQLite database
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(app.root_path, '../movie.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.app_context().push()

db = SQLAlchemy(app)

class DoubanMovie(db.Model):
    __tablename__ = 'douban_movies'

    movie_id = db.Column(db.Integer, primary_key=True)
    movie_name = db.Column(db.String(20))
    release_date = db.Column(db.String(15))
    country = db.Column(db.String(20))
    movie_type = db.Column(db.String(10))
    release_year = db.Column(db.Integer)
    description = db.Column(db.Text)
    douban_url = db.Column(db.String(255))
    poster  = db.Column(db.String(255))
    douban_rate  = db.Column(db.Float)
    rating_count = db.Column(db.Integer)
    
    def __repr__(self):
        return f'<Douban {self.movie_name}>' 

In [110]:
movies = pd.read_sql_table('douban_movies', con=db.engine)
movies['link'] = movies.apply(lambda x: f"""<a href="{x['douban_url']}">{x['douban_rate']}</a>""", axis=1)
movies['release_year'] = movies.release_year.fillna(0).astype(np.int64)
movies['short'] = movies.description.str.replace(r'\n| |\u3000', '', regex=True)
movies['short'] = movies['short'].apply(lambda x: x if len(x)<155 else x[:155]+'...')
movies['poster'] = movies.poster.apply(lambda x: f"""<img src="{x}", height="100px"></img>""")
movies['country'] = movies.country.str.split('/', expand=True)[0]
cols = ['movie_id', 'movie_name', 'release_year', 'movie_type',  'country', 'link', 'short', 'poster']
vars = dict(movie_list = movies[cols].values.tolist(), ensure_ascii=True)


In [142]:
movies.country.str.split('/', expand=True)[0]

0        英国 
1         美国
2      中国大陆 
3        法国 
4        美国 
       ...  
423     中国台湾
424       美国
425       美国
426       印度
427       韩国
Name: 0, Length: 428, dtype: object

In [117]:
movies.columns

Index(['movie_id', 'movie_name', 'release_date', 'country', 'movie_type',
       'release_year', 'description', 'douban_url', 'poster', 'douban_rate',
       'rating_count', 'link', 'short'],
      dtype='object')

In [138]:
def hbar_option(title, y_category, x_data):
    labels = [f"{y}, {x}" for y, x in zip(y_category, x_data)]
    return {
        'title': {'text': title},
        'xAxis': {
            'min': 7,
            'max': 10,
            'type': 'value',
            'position': 'top',
            'splitLine': {'lineStyle': {'type': 'dashed'}}
        },
        'yAxis': {
            'type': 'category',
            'inverse': True,
            'axisLine': {'show': False},
            'axisLabel': {'show': False},
            'axisTick': {'show': False},
            'splitLine': {'show': False},
            'data': labels
        },
        'series': [
            {
                'name': '',
                'type': 'bar',
                'label': {'show': True, 'formatter': '{b}'},
                'data': x_data
            }
        ]
    }


data = movies.loc[
    movies.country.str.contains('大陆'), 
    ['movie_name', 'douban_rate']
].sort_values('douban_rate', ascending=False)[:10].set_index('movie_name')

hbar_option("大陆电影高分榜", data.index.tolist(), data['douban_rate'].tolist())

{'title': {'text': '大陆电影高分榜'},
 'xAxis': {'min': 7,
  'max': 10,
  'type': 'value',
  'position': 'top',
  'splitLine': {'lineStyle': {'type': 'dashed'}}},
 'yAxis': {'type': 'category',
  'inverse': True,
  'axisLine': {'show': False},
  'axisLabel': {'show': False},
  'axisTick': {'show': False},
  'splitLine': {'show': False},
  'data': ['霸王别姬, 9.6',
   '大闹天宫, 9.4',
   '末代皇帝, 9.3',
   '鬼子来了, 9.3',
   '哪吒闹海, 9.2',
   '天书奇谭, 9.2',
   '我不是药神, 9.0',
   '让子弹飞, 9.0',
   '可可西里, 8.9',
   '阳光灿烂的日子, 8.8']},
 'series': [{'name': '',
   'type': 'bar',
   'label': {'show': True, 'formatter': '{b}'},
   'data': [9.6, 9.4, 9.3, 9.3, 9.2, 9.2, 9.0, 9.0, 8.9, 8.8]}]}

In [139]:
data = movies.loc[
    movies.country.str.contains('香港|台湾'), 
    ['movie_name', 'douban_rate']
].sort_values('douban_rate', ascending=False)[:10].set_index('movie_name')

hbar_option("港台电影高分榜", data.index.tolist(), data['douban_rate'].tolist())

{'title': {'text': '港台电影高分榜'},
 'xAxis': {'min': 7,
  'max': 10,
  'type': 'value',
  'position': 'top',
  'splitLine': {'lineStyle': {'type': 'dashed'}}},
 'yAxis': {'type': 'category',
  'inverse': True,
  'axisLine': {'show': False},
  'axisLabel': {'show': False},
  'axisTick': {'show': False},
  'splitLine': {'show': False},
  'data': ['霸王别姬, 9.6',
   '无间道, 9.3',
   '饮食男女, 9.2',
   '少年派的奇幻漂流, 9.1',
   '让子弹飞, 9.0',
   '喜宴, 9.0',
   '春光乍泄, 9.0',
   '可可西里, 8.9',
   '牯岭街少年杀人事件, 8.9',
   '花样年华, 8.8']},
 'series': [{'name': '',
   'type': 'bar',
   'label': {'show': True, 'formatter': '{b}'},
   'data': [9.6, 9.3, 9.2, 9.1, 9.0, 9.0, 9.0, 8.9, 8.9, 8.8]}]}